#### Statistics Citations and Abtstracts

In [1]:
## spark version
# import os
# import sys

# spark_home = os.environ['SPARK_HOME']
# sys.path.insert(0, spark_home + "/python/")
# sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))

# # Initialize PySpark to predefine the SparkContext variable 'sc'
# execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

# papers = sc.textFile('SCC2015-metadata/paperMeta.txt')
# papers.take(2)

In [2]:
## ... copious amounts of logging info
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
rootLogger = logging.getLogger()
rootLogger.setLevel(logging.INFO)


In [4]:

import re
import csv
 
contractions = re.compile(r"'|-|\"}{")
# all non alphanumeric
symbols = re.compile(r'(\W+)', re.U)
# single character removal
singles = re.compile(r'(\s\S\s)', re.I|re.U)
# separators (any whitespace)
seps = re.compile(r'\s+')
 
# cleaner (order matters)
def clean(text):
    text = text.lower()
    text = contractions.sub('', text)
    text = symbols.sub(r' \1 ', text)
    text = singles.sub(' ', text)
    text = seps.sub(' ', text)
    return text
 
# sentence splitter
alteos = re.compile(r'([!\?])')
def sentences(l):
    l = alteos.sub(r' \1 .', l).rstrip("(\.)*\n")
    return l.split(".")
 
def abstractReader():
    with open('SCC2015-metadata/paperMeta.txt', 'r') as f:
        reader = csv.reader(f)
        header = reader.next()
        for l in reader:
            text = l[2] + '.  '  + l[4]
            text = [clean(s).split() for s in sentences(text)]
            wrds = [w for s in text for w in s]
            counter = wrds.count # minor speedup
            tkncnt = dict((w,counter(w)) for w in set(wrds))
            yield {'doi':l[0],'year':int(l[1]),'cites':int(l[3]),'text':tkncnt}


In [7]:
abstracts = abstractReader()
xout = open('statsAbsTkns.txt', 'w')
i = 0
for a in abstracts:
    i+=1
    for w in a['text']:
        xout.write(u"{0}\t{1}\t{2}\n".format(a['doi'],w,a['text'][w]))